In [2]:
!pip install -q langchain-community langchain_openai langchain_chroma
!pip install -q gradio
!pip install -Uq langchain-huggingface
!pip install -q chromadb langchain-community


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 4.3 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 34.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.4/55.4 kB 5.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 611.1/611.1 kB 27.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 43.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 415.4/415.4 kB 27.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 44.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 284.2/284.2 kB 21.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.9/94.9 kB 11.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 53.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 4.9 MB/s eta 0:00:00

In [3]:
!unzip knowledge-base.zip
%ls

Archive:  knowledge-base.zip
   creating: knowledge-base/
   creating: knowledge-base/company/
  inflating: knowledge-base/company/about.md  
  inflating: knowledge-base/company/careers.md  
  inflating: knowledge-base/company/overview.md  
   creating: knowledge-base/contracts/
  inflating: knowledge-base/contracts/.md  
  inflating: knowledge-base/contracts/Contract with Apex Reinsurance for Rellm.md  
  inflating: knowledge-base/contracts/Contract with Belvedere Insurance for Markellm.md  
  inflating: knowledge-base/contracts/Contract with BrightWay Solutions for Markellm.md  
  inflating: knowledge-base/contracts/Contract with EverGuard Insurance for Rellm.md  
  inflating: knowledge-base/contracts/Contract with GreenField Holdings for Markellm.md  
  inflating: knowledge-base/contracts/Contract with Greenstone Insurance for Homellm.md  
  inflating: knowledge-base/contracts/Contract with GreenValley Insurance for Homellm.md  
  inflating: knowledge-base/contracts/Contract with Pi

In [4]:
import os
import glob
import gradio as gr
import langchain_chroma as Chroma
from langchain.document_loaders import DirectoryLoader, TextLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_community.vectorstores import Chroma
import numpy as np
from langchain.memory import ConversationBufferMemory
from langchain.chains import ConversationalRetrievalChain
from openai import OpenAI
from langchain_core.language_models import LLM
from langchain_core.outputs import LLMResult

MODEL = "gpt-4o"
db_name = "vector_chroma_db"

In [5]:
# Get API keys

from google.colab import userdata
gpt_4o_api_key=userdata.get('gpt-4o')


In [6]:
# Read in documents using LangChain's loaders

folders = glob.glob("/content/knowledge-base/*")

text_loader_kwargs = {'encoding': 'utf-8'}

documents = []
for folder in folders:
    doc_type = os.path.basename(folder)
    loader = DirectoryLoader(folder, glob="**/*.md", loader_cls=TextLoader, loader_kwargs=text_loader_kwargs)
    folder_docs = loader.load()
    for doc in folder_docs:
        doc.metadata["doc_type"] = doc_type
        documents.append(doc)

In [7]:
len(documents)

31

In [8]:
# Split documents in chunks
text_splitter = RecursiveCharacterTextSplitter(chunk_size=800, chunk_overlap=150)
chunks = text_splitter.split_documents(documents)

In [9]:
# Specify document type to chunks

doc_types = set(chunk.metadata['doc_type'] for chunk in chunks)
print(f"Document types found: {', '.join(doc_types)}")

Document types found: company, contracts, products, employees


In [10]:
print(len(chunks))

152


In [11]:
# Import embedding model  from HF
embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")


# Delete anything if already exists
if os.path.exists(db_name):
    Chroma(persist_directory=db_name, embedding_function=embeddings).delete_collection()

# Create vectorstore
vectorstore = Chroma.from_documents(documents=chunks, embedding=embeddings, persist_directory=db_name)
print(f"Vectorstore created with {vectorstore._collection.count()} documents")

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.5k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

Vectorstore created with 152 documents


In [12]:
# Check dimensions of a vector

collection = vectorstore._collection
sample_embedding = collection.get(limit=1, include=["embeddings"])["embeddings"][0]
dimensions = len(sample_embedding)
print(f"The vectors have {dimensions:,} dimensions")

The vectors have 384 dimensions


In [13]:
client = OpenAI(
base_url="https://models.inference.ai.azure.com",
    api_key=gpt_4o_api_key,
)

def azure_openai_llm(messages, temperature=0.9, model="gpt-4o"):
    response = client.chat.completions.create(
        model=model,
        messages=messages,
        temperature=temperature
    )
    return response.choices[0].message.content

# Define a proper LangChain-compatible LLM wrapper
class AzureOpenAILLM(LLM):
    model: str = "gpt-4o-mini"
    temperature: float = 0.9

    def _call(self, prompt: str, stop=None) -> str:
        messages = [{"role": "user", "content": prompt}]
        return azure_openai_llm(messages, temperature=self.temperature, model=self.model)

    def _llm_type(self) -> str:
        return "azure_openai"

# Instantiate custom LLM wrapper
llm = AzureOpenAILLM()

# Store past conversations
memory = ConversationBufferMemory(memory_key='chat_history', return_messages=True)

# Retriever (from Chroma DB)
retriever = vectorstore.as_retriever(search_type='mmr',search_kwargs={"k": 10})

# Create ConversationalRetrievalChain using the custom LLM
conversation_chain = ConversationalRetrievalChain.from_llm(llm=llm, retriever=retriever, memory=memory)


<ipython-input-13-d5b489ad3f0d>:30: LangChainDeprecationWarning: Please see the migration guide at: https://python.langchain.com/docs/versions/migrating_memory/
  memory = ConversationBufferMemory(memory_key='chat_history', return_messages=True)


In [14]:
# Wrapping in a function

def chat(message, history):
    result = conversation_chain.invoke({"question": message})
    return result["answer"]
# Use Gradio:
view = gr.ChatInterface(chat, type="messages").launch(inbrowser=True)

Running Gradio in a Colab notebook requires sharing enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://e752b21f8ccfce480d.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
